# Precision-Preserving Ensemble

Fix the precision truncation bug by preserving original string values from source files.

In [ ]:
import pandas as pd
import numpy as np
import math
from numba import njit
import json
import os
from shapely.geometry import Polygon
from shapely import affinity
from collections import defaultdict

# Tree vertices
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

In [ ]:
@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a single N configuration"""
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx = min(mnx, X)
            mxx = max(mxx, X)
            mny = min(mny, Y)
            mxy = max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

In [ ]:
def parse_submission_with_strings(filepath):
    """Parse submission CSV and return dict of N -> DataFrame with original strings"""
    df = pd.read_csv(filepath)
    
    # Parse float values for scoring (but keep original strings)
    df['x_val'] = df['x'].str.replace('s', '').astype(float)
    df['y_val'] = df['y'].str.replace('s', '').astype(float)
    df['deg_val'] = df['deg'].str.replace('s', '').astype(float)
    
    # Extract N and tree_idx from id (format: NNN_idx)
    df['N'] = df['id'].str.split('_').str[0].astype(int)
    df['tree_idx'] = df['id'].str.split('_').str[1].astype(int)
    
    # Return dict of N -> DataFrame (with original strings preserved)
    configs = {}
    for n, group in df.groupby('N'):
        configs[n] = group.copy()
    
    return configs

In [ ]:
def calculate_score_from_df(df, tx, ty):
    """Calculate score from DataFrame"""
    xs = df['x_val'].values
    ys = df['y_val'].values
    degs = df['deg_val'].values
    return score_group(xs, ys, degs, tx, ty)

In [ ]:
def check_overlaps_strict(df, buffer_dist=1e-9):
    """Check for overlaps with strict tolerance"""
    n = len(df)
    if n <= 1:
        return True, []
    
    xs = df['x_val'].values
    ys = df['y_val'].values
    degs = df['deg_val'].values
    
    # Create polygons
    polygons = []
    for i in range(n):
        p = Polygon(zip(TX, TY))
        p = affinity.rotate(p, degs[i], origin=(0,0))
        p = affinity.translate(p, xs[i], ys[i])
        polygons.append(p)
    
    overlaps = []
    for i in range(n):
        for j in range(i+1, n):
            if polygons[i].intersects(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > 1e-12:  # Non-trivial overlap
                    overlaps.append((i, j, intersection.area))
    
    return len(overlaps) == 0, overlaps

In [ ]:
# Load baseline (known valid)
baseline_path = '/home/code/preoptimized/submission.csv'
baseline_configs = parse_submission_with_strings(baseline_path)

# Calculate baseline scores
baseline_scores = {}
baseline_total = 0
for n in range(1, 201):
    score = calculate_score_from_df(baseline_configs[n], TX, TY)
    baseline_scores[n] = score
    baseline_total += score

print(f"Baseline score: {baseline_total:.6f}")

# Verify baseline N=2 precision
print(f"\nBaseline N=2 x values:")
for _, row in baseline_configs[2].iterrows():
    print(f"  {row['x']}")

In [ ]:
# Scan all snapshot directories for submission files
snapshot_base = '/home/nonroot/snapshots/santa-2025/'
snapshot_dirs = os.listdir(snapshot_base)

all_sources = {}
all_sources['baseline'] = baseline_configs

print(f"Scanning {len(snapshot_dirs)} snapshot directories...")

for snap_dir in sorted(snapshot_dirs):
    snap_path = os.path.join(snapshot_base, snap_dir)
    
    # Check for submission.csv in various locations
    possible_paths = [
        os.path.join(snap_path, 'submission', 'submission.csv'),
        os.path.join(snap_path, 'code', 'submission.csv'),
    ]
    
    for path in possible_paths:
        if os.path.exists(path):
            try:
                configs = parse_submission_with_strings(path)
                if len(configs) == 200:  # Valid submission has 200 N values
                    all_sources[f'snap_{snap_dir}'] = configs
            except Exception as e:
                pass
            break

print(f"Loaded {len(all_sources)} sources")

In [ ]:
# For each N, find the best configuration that passes strict overlap validation
print("\nFinding best valid configuration for each N (preserving precision)...")

best_configs = {}  # N -> DataFrame with original strings
best_scores_by_n = {}
source_used = {}
validation_failures = []

for n in range(1, 201):
    best_score = float('inf')
    best_df = None
    best_source = None
    
    # Collect all candidates for this N
    candidates = []
    for source_name, configs in all_sources.items():
        if n in configs:
            df = configs[n]
            score = calculate_score_from_df(df, TX, TY)
            candidates.append((score, source_name, df))
    
    # Sort by score (best first)
    candidates.sort(key=lambda x: x[0])
    
    # Try each candidate until we find one that passes validation
    for score, source_name, df in candidates:
        valid, overlaps = check_overlaps_strict(df)
        
        if valid:
            best_score = score
            best_df = df
            best_source = source_name
            break
        else:
            if source_name != 'baseline':
                validation_failures.append((n, source_name, score, len(overlaps)))
    
    # If no valid config found, use baseline (should always be valid)
    if best_df is None:
        best_df = baseline_configs[n]
        best_score = baseline_scores[n]
        best_source = 'baseline_fallback'
    
    best_configs[n] = best_df
    best_scores_by_n[n] = best_score
    source_used[n] = best_source

print(f"\nValidation failures (better score but overlapping): {len(validation_failures)}")

In [ ]:
# Calculate final score
final_score = sum(best_scores_by_n.values())
print(f"\nFinal ensemble score: {final_score:.6f}")
print(f"Baseline score: {baseline_total:.6f}")
print(f"Improvement: {baseline_total - final_score:.6f}")

# Source breakdown
source_counts = defaultdict(int)
for n, source in source_used.items():
    source_counts[source] += 1

print(f"\nSource breakdown:")
for source, count in sorted(source_counts.items(), key=lambda x: -x[1])[:10]:
    print(f"  {source}: {count} N values")

In [ ]:
# Generate submission CSV PRESERVING ORIGINAL PRECISION
def generate_submission_preserve_precision(configs_by_n, output_path):
    """Generate submission preserving original string precision"""
    rows = []
    for n in range(1, 201):
        df = configs_by_n[n]
        for _, row in df.iterrows():
            rows.append({
                'id': f'{n:03d}_{row["tree_idx"]}',
                'x': row['x'],  # Keep original string!
                'y': row['y'],  # Keep original string!
                'deg': row['deg']  # Keep original string!
            })
    result_df = pd.DataFrame(rows)
    result_df.to_csv(output_path, index=False)
    return result_df

# Save submission
os.makedirs('/home/submission', exist_ok=True)
submission_df = generate_submission_preserve_precision(best_configs, '/home/submission/submission.csv')
print(f"Saved submission to /home/submission/submission.csv")
print(f"Total rows: {len(submission_df)}")

# Verify precision is preserved
print(f"\nVerifying N=2 precision in saved submission:")
for _, row in submission_df[submission_df['id'].str.startswith('002_')].iterrows():
    print(f"  {row['id']}: x={row['x']}")

In [ ]:
# Verify the saved submission score
verify_configs = parse_submission_with_strings('/home/submission/submission.csv')
verify_total = 0
for n in range(1, 201):
    score = calculate_score_from_df(verify_configs[n], TX, TY)
    verify_total += score

print(f"\nVerification of saved submission: {verify_total:.6f}")
print(f"Expected: {final_score:.6f}")
print(f"Match: {abs(verify_total - final_score) < 1e-6}")

In [ ]:
# Save metrics
metrics = {
    'cv_score': verify_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - verify_total,
    'validation_failures': len(validation_failures)
}

with open('/home/code/experiments/004_precision_fix/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nSaved metrics to experiments/004_precision_fix/metrics.json")
print(f"CV Score: {verify_total:.6f}")